# First Look at the Data

## Table of Contents
* [Import Libraries](#chapter1)
    * [Import magic autoreload](#section_1_1)
    * [Import the libraries](#section_1_2)
    * [Import custom functions.](#section_1_3)
* [Read the Data](#chapter2)
* [Inspect the Data](#chapter3)
* [Create new variables](chapter4)
* [Save the Data](#chapter5)


## Import Libraries: <a class="anchor" id="chapter1"></a>

Import the autoreload extension so that any changes in external python modules are automatically loaded. <a class="anchor" id="section_1_1"></a>

Import the libraries we will use in this notebook. <a class="anchor" id="section_1_2"></a>

Set the current working directory to the project folder.

Import our custom functions. <a class="anchor" id="section_1_3"></a>

In [3]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("C:/Users/migue/OneDrive - NOVAIMS/Data Science/Coding Courses/Machine Learning II/Project")
# wd stands for working directory
wd = os.getcwd()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from functions.funcs import *

# Read the data: <a class="anchor" id="chapter2"></a>

List the files in the directory. 

In [4]:
dfs = create_dfs(wd + "\\others&backup\\prof_data\\")

Created dataframe Basket from Customer Basket Dataset.csv
Created dataframe Info from Customer Info Dataset.csv
Created dataframe Mapping from Product Mapping Excel File.xlsx
File Project Description and Info.pdf is not a .csv or .xlsx file. Skipping it.


Create dataframes from the csv files. <a class="anchor" id="section_2_1"></a>

In [5]:
# Create a global variable for each dataframe in the dfs dict
for key in dfs.keys():
    globals()[key] = dfs[key]
    print(f"Created global variable {blue}{key}{end} with values from dictionary dfs's key {blue}{key}{end}")

Created global variable Basket with values from dictionary dfs's key Basket
Created global variable Info with values from dictionary dfs's key Info
Created global variable Mapping with values from dictionary dfs's key Mapping


# Inspect the data: <a class="anchor" id="chapter3"></a>

In [6]:
print_cols(Basket, "Basket")
print_cols(Info, "Info")
print_cols(Mapping, "Mapping")

Columns in Basket are: 
	-customer_id, invoice_id, list_of_goods

Columns in Info are: 
	-customer_id, customer_name, customer_gender, customer_birthdate, kids_home, 
	-teens_home, number_complaints, distinct_stores_visited, lifetime_spend_groceries, lifetime_spend_electronics, 
	-typical_hour, lifetime_spend_vegetables, lifetime_spend_nonalcohol_drinks, lifetime_spend_alcohol_drinks, lifetime_spend_meat, 
	-lifetime_spend_fish, lifetime_spend_hygiene, lifetime_spend_videogames, lifetime_total_distinct_products, percentage_of_products_bought_promotion, 
	-year_first_transaction, loyalty_card_number, latitude, longitude

Columns in Mapping are: 
	-product_name, category



In [7]:
print_na_cols(Basket, "Basket")
print_na_cols(Info, "Info")
print_na_cols(Mapping, "Mapping")

Basket has no missing values.

In Info, the following columns have missing values:
	-Column loyalty_card_number has 24175 missing values. This equals 80.58% of its values.
	
Mapping has no missing values.



In [8]:
# check for duplicates
print(f"Number of duplicates in {blue}Basket{end}: {red}{Basket.duplicated().sum()}{end}")
print(f"Number of duplicates in {blue}Info{end}: {red}{Info.duplicated().sum()}{end}")
print(f"Number of duplicates in {blue}Mapping{end}: {red}{Mapping.duplicated().sum()}{end}")

Number of duplicates in Basket: 0
Number of duplicates in Info: 0
Number of duplicates in Mapping: 1


In [9]:
# check for duplicate customer_id in Info
print(f"Number of duplicate customer_id in {blue}Info{end}: {red}{Info.duplicated(subset='customer_id').sum()}{end}")
print(f"Number of duplicate coordinates in {blue}Info{end}: {red}{Info.duplicated(subset=['latitude', 'longitude']).sum()}{end}")


Number of duplicate customer_id in Info: 0
Number of duplicate coordinates in Info: 0


In [10]:
# print the duplicate rows in mapping with their index
print(Mapping[Mapping.duplicated(keep=False)])

    product_name    category
128    asparagus  vegetables
135    asparagus  vegetables


In [11]:
# drop duplicates in Mapping
# Mapping.drop_duplicates(inplace=True)

In [12]:
print_inf_cols(Basket, "Basket")
print_inf_cols(Info, "Info")
print_inf_cols(Mapping, "Mapping")

Basket has no infinite values.

In Info, the following columns have infinite values:
	-Column typical_hour has 2 infinite values. This equals 0.01% of its values.
	-Column lifetime_spend_videogames has 226 infinite values. This equals 0.75% of its values.
	
Mapping has no infinite values.



Let's inspect the rows that have infinite values in the columns.

In [13]:
# print the number of rows that have inf values
Info[Info.isin([np.inf, -np.inf]).any(axis = 1)].shape

(226, 24)

We can see that all the customers with infinite values appear to be supermarkets. Let's check the number of supermarkets in the dataset.

In [14]:
# check number of supermarkets in the dataset (rows that have supermarket in the customer_name)
Info[Info.customer_name.str.contains("Supermarket")].shape

(226, 24)

In [15]:
# compare the two dataframes to see if they are the same: 
Info[Info.customer_name.str.contains("Supermarket")].equals(Info[Info.isin([np.inf, -np.inf]).any(axis = 1)])

True

We can now see that there are 226 supermarkets on the dataset and that these are the customers with infinite values. <br> We are going to replace these values with 0 as this error in data is simply because supermarkets don't buy videogames. <br>
We will also look into these customers in more detail.

In [16]:
# replace inf values in column lifetime_spend_videogames with 0
Info['lifetime_spend_videogames'].replace([np.inf, -np.inf], 0, inplace=True)

In [17]:
supermarkets = Info[Info.customer_name.str.contains("Supermarket", case = False)]

In [18]:
supermarkets.shape

(226, 24)

In [19]:
supermarkets['typical_hour'].describe()

count    226.000000
mean           -inf
std             NaN
min            -inf
25%        0.575646
50%        0.693147
75%        0.722593
max        0.783874
Name: typical_hour, dtype: float64

In [20]:
# impute inf values in typical hour Info due to being few and not having a clear pattern
Info.loc[Info['typical_hour'] == -np.inf, 'typical_hour'] = supermarkets.loc[supermarkets['typical_hour'] != -np.inf, 'typical_hour'].mean()
supermarkets.loc[supermarkets['typical_hour'] == -np.inf, 'typical_hour'] = supermarkets.loc[supermarkets['typical_hour'] != -np.inf, 'typical_hour'].mean()

In [21]:
supermarkets['typical_hour'].describe()

count    226.000000
mean       0.625775
std        0.154783
min        0.000000
25%        0.575646
50%        0.693147
75%        0.722593
max        0.783874
Name: typical_hour, dtype: float64

# Create new variables <a class="anchor" id="chapter4"></a>
Using information in the dataset we can create new variables that might be useful for our analysis.

### Create a new variable to tell if a customer is a business or not <a class="anchor" id="section_4_1"></a>

Let's create a new column with a boolean value indicating if the customer is a business or not.

In [22]:
Info['is_business'] = [1 if x == True else 0 for x in Info.customer_name.str.contains("Supermarket", case = False)]

Let's create another new column with a boolean value indicating if the customer has a loyalty card or not.

In [39]:
Info['has_loyalty_card'] = [0 if pd.isna(x) else 1 for x in Info.loyalty_card_number]

### Create a new column with the Age of each customer. <a class="anchor" id="section_4_2"></a>

In [23]:
Info['customer_birthdate'] = pd.to_datetime(Info['customer_birthdate'], format='%m/%d/%Y %H:%M %p')

In [24]:
# age is the difference between the current year and the year in customer_birthdate
Info['age'] = 2023 - Info.customer_birthdate.dt.year

### Column with the time the customer has existed. This will be *years_as_customer*. <a class="anchor" id="section_4_3"></a>

In [25]:
print_cols(Info, "Info")

Columns in Info are: 
	-customer_id, customer_name, customer_gender, customer_birthdate, kids_home, 
	-teens_home, number_complaints, distinct_stores_visited, lifetime_spend_groceries, lifetime_spend_electronics, 
	-typical_hour, lifetime_spend_vegetables, lifetime_spend_nonalcohol_drinks, lifetime_spend_alcohol_drinks, lifetime_spend_meat, 
	-lifetime_spend_fish, lifetime_spend_hygiene, lifetime_spend_videogames, lifetime_total_distinct_products, percentage_of_products_bought_promotion, 
	-year_first_transaction, loyalty_card_number, latitude, longitude, is_business, age



This next cell gives us the last year that we have reported a new customer, so we will use that +1 as the current year. <br>
The reason for the +1 is to avoid having a customer with 0 years as a customer. <br>
This will in turn avoid having a division by 0 when we calculate the *years_as_customer*.

In [26]:
current_year = Info['year_first_transaction'].max() + 1

In [27]:
Info['years_as_customer'] = current_year - Info['year_first_transaction']

In [28]:
Info['years_as_customer'].max()

32.0

In [29]:
# get the row with the max value in years_as_customer
Info[['age', 'years_as_customer', 'year_first_transaction']][Info['years_as_customer'] >= Info['age'] - 18]

,age,years_as_customer,year_first_transaction
4,23,6.0,2015.0
6,28,10.0,2011.0
10,35,21.0,2000.0
17,28,18.0,2003.0
22,22,4.0,2017.0
...,...,...,...
29950,26,17.0,2004.0
29975,28,11.0,2010.0
29978,25,11.0,2010.0
29984,22,4.0,2017.0


### Create a new column with the lifetime spent by the customer <a class="anchor" id="section_4_4"></a>

Using all the columns in the dataset with *lifetime_spend* in their name, we can create a new variable that represents the total amount of money they spent.

In [30]:
# create new column with total amount spent by each customer by summing all lifetime_spent columns
Info['lifetime_spend'] = Info[[col for col in Info.columns if 'lifetime_spend' in col]].sum(axis=1)

In [31]:
Info['lifetime_spend'].describe()

count    30000.000000
mean      8506.864009
std       7031.387462
min        543.000000
25%       2431.750000
50%       9170.000000
75%      11137.000000
max      36258.059197
Name: lifetime_spend, dtype: float64

### Create a new variable that represents the total amount of money spent per year <a class="anchor" id="section_4_5"></a>

We can do this by dividing the total amount of money spent by the number of years they have been a customer.

In [32]:
Info['spend_per_year'] = Info['lifetime_spend'] / Info['years_as_customer']

In [33]:
Info['spend_per_year'].describe()

count    30000.000000
mean       695.863268
std        563.651525
min         38.000000
25%        223.653846
50%        724.928205
75%        996.117424
max      10975.000000
Name: spend_per_year, dtype: float64

In [34]:
# max info spend per year
Info[Info['spend_per_year'] == Info['spend_per_year'].max()]

,customer_id,customer_name,customer_gender,customer_birthdate,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,...,percentage_of_products_bought_promotion,year_first_transaction,loyalty_card_number,latitude,longitude,is_business,age,years_as_customer,lifetime_spend,spend_per_year
24595,23396,Aurora Hartl,female,1953-09-11 01:34:00,6.0,0.0,1.0,3.0,5315.0,211.0,...,0.206432,2020.0,NaN,38.731795,-9.163617,0,70,1.0,10975.0,10975.0


### Create variable with the education level of the customers <a class="anchor" id="section_4_6"></a>

In [35]:
# create variables with the education level of the customers, None, Bsc, Msc and PhD. These will be 0, 1, 2 and 3 respectively.
# The information regarding this is in the customer_name column of the Info dataframe
Info['education_level'] = create_educ_level(Info)

In [36]:
Info['education_level'].value_counts()

0    18568
1     3815
3     3810
2     3807
Name: education_level, dtype: int64

## Save the Data <a class="anchor" id="chapter5"></a>
This saves the current state of the data to new csv files so that we can use them in other notebooks.

In [37]:
# save all three dataframes to csv files in a new folder named treated in the data folder of the project
save_to_csv(dfs, wd + "/data/")